In [55]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

html_data = None

html_path = "VETA5_98-CC-E4-00-2D-E8_2023-10-30_14-19-29_Service/VETA5_98-CC-E4-00-2D-E8_2023-10-30_14-19-29_Service.html"

with open(html_path,"r") as html_source:
    html_data = html_source.readlines()

soup = BeautifulSoup("\n".join(html_data),"html.parser")



In [12]:
#print(soup.prettify())

In [51]:
rawJ = soup.find_all("script")
#for i in range(len(rawJ)):
#    print(i)
#    print(rawJ[i])

J = str(rawJ[0])
J1 = J.split('var T_Data3 = ["')

In [52]:
raw_df = J1[1].split(",];")[0].replace("<br>"," ")
cells = raw_df.split('","')
N = len(cells)
n_col = 7
n_row = N // n_col

(n_row,n_col)

_data = {}

for i in range(n_col):
    idx = i*n_row
    _data[cells[idx]] = []
    for j in range(1,n_row):
        _data[cells[idx]].append(cells[idx+j])


In [83]:
df = pd.DataFrame(_data)
df["End Case"] = df["Date Time Event"].apply(lambda x: "End Case" in str(x))
df["Date Time Event"] = df["Date Time Event"].apply(lambda x: datetime.strptime(str(x).replace("End Case",""),"%Y-%m-%d %H:%M "))

In [65]:
df.head()

,Date Time Event,Fi/Et CO2(Hgmm),PEAK(cmH2O),PEEP(cmH2O),Vt(ml),MV(l/min),RR(bpm),End Case
0,2023-10-27 18:32:00,,,,,,,True
1,2023-10-27 18:32:00,0/38,16,3,350,4.4,12,False
2,2023-10-27 18:31:00,0/41,16,3,379,4.5,12,False
3,2023-10-27 18:30:00,0/42,16,3,381,4.6,12,False
4,2023-10-27 18:29:00,0/40,---,---,---,---,7,False


In [84]:
case_end_rows = df[df["End Case"]== True]
case_ends = case_end_rows.sort_values(by="Date Time Event")["Date Time Event"].tolist()
case_ends

[Timestamp('2023-10-25 12:17:00'),
 Timestamp('2023-10-25 16:25:00'),
 Timestamp('2023-10-25 20:20:00'),
 Timestamp('2023-10-26 10:27:00'),
 Timestamp('2023-10-26 14:35:00'),
 Timestamp('2023-10-26 21:12:00'),
 Timestamp('2023-10-27 10:42:00'),
 Timestamp('2023-10-27 14:36:00'),
 Timestamp('2023-10-27 18:32:00')]

In [85]:
case_dict = {}
for ce in case_ends:
    case_dict[ce] = df[df["Date Time Event"]<=ce]
    df = df.drop(case_dict[ce].index)
    case_dict[ce] = case_dict[ce].reset_index(drop=True)[1:].sort_values(by="Date Time Event").reset_index(drop=True)

In [86]:
case_dict

{Timestamp('2023-10-25 12:17:00'):         Date Time Event Fi/Et CO2(Hgmm) PEAK(cmH2O) PEEP(cmH2O) Vt(ml)  \
 0   2023-10-25 07:10:00             0/0           1           1    245   
 1   2023-10-25 07:11:00            0/39          14           2    322   
 2   2023-10-25 07:12:00            0/43          16           3    320   
 3   2023-10-25 07:13:00            0/41          16           4    329   
 4   2023-10-25 07:14:00            0/40          16           3    323   
 ..                  ...             ...         ...         ...    ...   
 303 2023-10-25 12:13:00            0/41          15           4    316   
 304 2023-10-25 12:14:00            0/41          15           4    322   
 305 2023-10-25 12:15:00            0/40          15           4    319   
 306 2023-10-25 12:16:00            0/41          15           3    317   
 307 2023-10-25 12:17:00            0/42          15           3    319   
 
     MV(l/min) RR(bpm)  End Case  
 0         2.4     20"     Fa

In [90]:
for ce, c in case_dict.items():
    start = c["Date Time Event"].min()
    end = ce
    cols = c.columns[:-1]
    out_path = f"{start.strftime('%Y-%m-%d_%H-%M')}--to--{end.strftime('%Y-%m-%d_%H-%M')}.csv"
    c[cols].to_csv(out_path,index=False)